In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"

!nvidia-smi

Wed Jun 19 17:23:09 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4090        Off |   00000000:19:00.0 Off |                  Off |
|  0%   49C    P8             27W /  450W |      11MiB /  24564MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
import torch
from diffusers import StableDiffusionPipeline, DDPMScheduler, DPMSolverSDEScheduler

pipe = StableDiffusionPipeline.from_single_file("/data/sd_files/checkpoint/beautifulRealistic_v7.safetensors",
                                                torch_dtype=torch.float16, use_safetensors=True, variant="fp16")

pipe.to("cuda")
pipe.scheduler = DPMSolverSDEScheduler.from_config(pipe.scheduler.config)

print('done')

/home/scpark/anaconda3/envs/ste/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-06-19 17:23:11.703005: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-19 17:23:12.613006: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/scpark/anaconda3/envs/ste/lib/python3.9/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access Untyped

done


/home/scpark/anaconda3/envs/ste/lib/python3.9/site-packages/diffusers/configuration_utils.py:139: FutureWarning: Accessing config attribute `use_karras_sigmas` directly via 'DPMSolverSDEScheduler' object attribute is deprecated. Please access 'use_karras_sigmas' over 'DPMSolverSDEScheduler's config object instead, e.g. 'scheduler.config.use_karras_sigmas'.
  deprecate("direct config name access", "1.0.0", deprecation_message, standard_warn=False)


In [3]:
import os

def read_txt_files_in_directory(directory_path):
    all_files_content = []

    # 디렉토리 내 모든 파일을 확인
    for filename in os.listdir(directory_path):
        # .txt 파일만 처리
        if filename.endswith(".txt"):
            file_path = os.path.join(directory_path, filename)
            with open(file_path, 'r', encoding='utf-8') as f:
                file_content = f.readline().strip()
                all_files_content.append(file_content.replace('jisoo, ', ''))

    return all_files_content

# 사용 예제
directory_path = "/data/sd-dataset/shark_png"
files_content = read_txt_files_in_directory(directory_path)

new_files_content = []
for content in files_content:
    new_files_content.append(content.replace('mychar4', 'shark'))

files_content = new_files_content
files_content

['shark, a cartoon shark with a big smile',
 'shark, a cartoon shark with a big smile',
 'shark, a yellow and white stuffed shark with a big smile',
 'shark, a cartoon shark with a big smile on its face',
 'shark, a cartoon shark with numbers on it']

In [ ]:
import os
save_dir = '/data/sd-dataset/shark_prior'
os.makedirs(save_dir, exist_ok=True)

from tqdm import tqdm
import numpy as np

for i in tqdm(range(500)):
    prompt = files_content[np.random.randint(0, len(files_content))]
    image = pipe(prompt=prompt,
                 height=512,
                 width=512,
                 num_inference_steps=20,
                 guidance_scale=7,
                ).images[0]
    image.save(os.path.join(save_dir, str(i) + '.png'))
    
    with open(os.path.join(save_dir, str(i) + '.txt'), 'w') as f:
        f.write(prompt)


 90%|██████████████████████████████████████▋    | 18/20 [00:02<00:00,  9.68it/s]/home/scpark/anaconda3/envs/ste/lib/python3.9/site-packages/torchsde/_brownian/brownian_interval.py:599: UserWarning: Should have ta>=t0 but got ta=0.0291675366461277 and t0=0.029168.
  warnings.warn(f"Should have ta>=t0 but got ta={ta} and t0={self._start}.")

 85%|████████████████████████████████████▌      | 17/20 [00:01<00:00, 10.05it/s]


 10%|████▍                                       | 2/20 [00:00<00:01, 13.37it/s]

In [7]:
print('done')

done
